In [91]:
import pandas as pd # type: ignore


In [13]:
df = pd.read_csv('bible2.csv') # csv 파일을 import하는 부분

In [22]:
df.shape # 총 31138개의 row와 11개의 column이 있다.

(31138, 11)

In [23]:
df.head() # 처음 5개의 row를 보여준다.

,idx,cate,book,chapter,paragraph,sentence,testament,long_label,short_label,count,endOfChapter
0,1,1,1,1,1,<천지 창조> 태초에 하나님이 천지를 창조하시니라,구,창세기,창,27,0
1,2,1,1,1,2,땅이 혼돈하고 공허하며 흑암이 깊음 위에 있고 하나님의 영은 수면 위에 운행하시니라,구,창세기,창,46,0
2,3,1,1,1,3,하나님이 이르시되 빛이 있으라 하시니 빛이 있었고,구,창세기,창,27,0
3,4,1,1,1,4,빛이 하나님이 보시기에 좋았더라 하나님이 빛과 어둠을 나누사,구,창세기,창,33,0
4,5,1,1,1,5,하나님이 빛을 낮이라 부르시고 어둠을 밤이라 부르시니라 저녁이 되고 아침이 되니 이...,구,창세기,창,55,0


In [33]:
new_df = df.copy() # df를 복사하여 new_df에 저장한다.

In [93]:
new_df['acc_count'] = new_df['count'].cumsum() # 각 절의 글자수 column의 누적합을 acc_count column에 저장한다.

In [37]:
new_df.tail() # 마지막 5개의 row를 보여준다.

,idx,cate,book,chapter,paragraph,sentence,testament,long_label,short_label,count,endOfChapter,acc_count
31133,31134,2,66,22,17,성령과 신부가 말씀하시기를 오라 하시는도다 듣는 자도 오라 할 것이요 목마른 자도 ...,신,요한계시록,계,78,0,1818469
31134,31135,2,66,22,18,내가 이 두루마리의 예언의 말씀을 듣는 모든 사람에게 증언하노니 만일 누구든지 이것...,신,요한계시록,계,88,0,1818557
31135,31136,2,66,22,19,만일 누구든지 이 두루마리의 예언의 말씀에서 제하여 버리면 하나님이 이 두루마리에 ...,신,요한계시록,계,79,0,1818636
31136,31137,2,66,22,20,이것들을 증언하신 이가 이르시되 내가 진실로 속히 오리라 하시거늘 아멘 주 예수여 ...,신,요한계시록,계,51,0,1818687
31137,31138,2,66,22,21,주 예수의 은혜가 모든 자들에게 있을지어다 아멘,신,요한계시록,계,26,0,1818713


In [94]:
new_df['count'].sum() # 절의 글자수 column의 총합을 구한다. 1818713개 이다.

np.int64(1818713)

In [95]:
chapter_counts = new_df.groupby('long_label', as_index=False, sort=False).agg({'count': 'sum'}).sort_index() # long_label을 기준으로 글자수 column의 합을 구한다.
chapter_counts

,long_label,count
0,창세기,85853
1,출애굽기,68814
2,레위기,48991
3,민수기,69581
4,신명기,62885
...,...,...
61,요한1서,6487
62,요한2서,760
63,요한3서,811
64,유다서,1686


In [41]:
round(new_df['acc_count'].max()/180) # 마지막 row의 누적합 1818713을 180으로 나누어 1일당 읽어야 하는 글자 수를 구한다. 10104개 이다.

10104

In [96]:
new_df[new_df['acc_count'] <= 10104].iloc[[0]] # 1일차에 읽어야 절을 표시한다.

,idx,cate,book,chapter,paragraph,sentence,testament,long_label,short_label,count,endOfChapter,acc_count
0,1,1,1,1,1,<천지 창조> 태초에 하나님이 천지를 창조하시니라,구,창세기,창,27,0,27


In [50]:
new_df[new_df['acc_count'] <= 10104].iloc[[-1]] # 2일차에 읽어야 절을 표시한다.

,idx,cate,book,chapter,paragraph,sentence,testament,long_label,short_label,count,endOfChapter,acc_count
206,207,1,1,9,1,<하나님이 노아와 언약을 세우시다> 하나님이 노아와 그 아들들에게 복을 주시며 그들...,구,창세기,창,71,0,10064


In [49]:
new_df[new_df['acc_count'] <= 20208].iloc[[-1]] # 3일차에 읽어야 절을 표시한다.

,idx,cate,book,chapter,paragraph,sentence,testament,long_label,short_label,count,endOfChapter,acc_count
412,413,1,1,17,15,하나님이 또 아브라함에게 이르시되 네 아내 사래는 이름을 사래라 하지 말고 사라라 하라,구,창세기,창,48,0,20198


In [97]:
def get_full_reading_info_df(day): # day를 인자로 받아서 day일차에 읽어야 하는 절을 표시하는 함수를 만든다.
  _list = [] # 빈 리스트를 만든다.
  unit_of_reading = round(new_df['acc_count'].max()/day) # 1일당 읽어야 하는 글자 수를 구한다.
  _list.append(new_df.iloc[[0]].index.values[0]) # 첫 날에 읽어야 하는 절을 리스트에 추가한다.
  for i in range(1, day): # 1부터 day까지 반복한다.
    _list.append(new_df[new_df['acc_count'] <= unit_of_reading * i].iloc[[-1]].index.values[0]) # day일차에 읽어야 하는 절을 리스트에 추가한다.
  result_df = new_df.loc[_list] # 리스트에 있는 index를 이용하여 해당 row를 가져와서 result_df에 저장한다.
  return result_df # result_df를 반환한다.
  

In [98]:
get_full_reading_info_df(180) # 180일 동안 읽어야 하는 절을 표시한다.

,idx,cate,book,chapter,paragraph,sentence,testament,long_label,short_label,count,endOfChapter,acc_count
0,1,1,1,1,1,<천지 창조> 태초에 하나님이 천지를 창조하시니라,구,창세기,창,27,0,27
206,207,1,1,9,1,<하나님이 노아와 언약을 세우시다> 하나님이 노아와 그 아들들에게 복을 주시며 그들...,구,창세기,창,71,0,10064
412,413,1,1,17,15,하나님이 또 아브라함에게 이르시되 네 아내 사래는 이름을 사래라 하지 말고 사라라 하라,구,창세기,창,48,0,20198
581,582,1,1,23,10,에브론이 헷 족속 중에 앉아 있더니 그가 헷 족속 곧 성문에 들어온 모든 자가 듣는...,구,창세기,창,65,0,30288
758,759,1,1,27,31,그가 별미를 만들어 아버지에게로 가지고 가서 이르되 아버지여 일어나서 아들이 사냥한...,구,창세기,창,68,0,40373
...,...,...,...,...,...,...,...,...,...,...,...,...
30327,30328,2,59,1,26,누구든지 스스로 경건하다 생각하며 자기 혀를 재갈 물리지 아니하고 자기 마음을 속이...,신,야고보서,약,62,0,1768142
30501,30502,2,60,5,1,<하나님의 양 무리를 치라> 너희 중 장로들에게 권하노니 나는 함께 장로 된 자요 ...,신,베드로전서,벧전,76,0,1778251
30664,30665,2,62,5,5,예수께서 하나님의 아들이심을 믿는 자가 아니면 세상을 이기는 자가 누구냐,신,요한1서,요일,40,0,1788354
30823,30824,2,66,5,8,그 두루마리를 취하시매 네 생물과 이십사 장로들이 그 어린 양 앞에 엎드려 각각 거...,신,요한계시록,계,82,0,1798455


In [99]:
get_full_reading_info_df(360) # 360일 동안 읽어야 하는 절을 표시한다.

,idx,cate,book,chapter,paragraph,sentence,testament,long_label,short_label,count,endOfChapter,acc_count
0,1,1,1,1,1,<천지 창조> 태초에 하나님이 천지를 창조하시니라,구,창세기,창,27,0,27
91,92,1,1,4,12,네가 밭을 갈아도 땅이 다시는 그 효력을 네게 주지 아니할 것이요 너는 땅에서 피하...,구,창세기,창,59,0,5030
206,207,1,1,9,1,<하나님이 노아와 언약을 세우시다> 하나님이 노아와 그 아들들에게 복을 주시며 그들...,구,창세기,창,71,0,10064
323,324,1,1,13,5,아브람의 일행 롯도 양과 소와 장막이 있으므로,구,창세기,창,25,0,15155
412,413,1,1,17,15,하나님이 또 아브라함에게 이르시되 네 아내 사래는 이름을 사래라 하지 말고 사라라 하라,구,창세기,창,48,0,20198
...,...,...,...,...,...,...,...,...,...,...,...,...
30747,30748,2,66,1,14,그의 머리와 털의 희기가 흰 양털 같고 눈 같으며 그의 눈은 불꽃 같고,신,요한계시록,계,39,0,1793456
30823,30824,2,66,5,8,그 두루마리를 취하시매 네 생물과 이십사 장로들이 그 어린 양 앞에 엎드려 각각 거...,신,요한계시록,계,82,0,1798455
30904,30905,2,66,10,7,일곱째 천사가 소리 내는 날 그의 나팔을 불려고 할 때에 하나님이 그의 종 선지자들...,신,요한계시록,계,81,0,1803501
30979,30980,2,66,14,17,<마지막 재난을 가지고 온 천사> 또 다른 천사가 하늘에 있는 성전에서 나오는데 역...,신,요한계시록,계,59,0,1808557
